In [ ]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

_ = load_dotenv()

In [ ]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DOC_PATH = "/Users/martijnbeeks/Downloads/TN459590 - EF29 Aankondiging gegunde opdracht - algemene richtlijn, standaardregeling 20240401142856.pdf"
CHROMA_PATH = "/Users/martijnbeeks/Development/Personal/tendergpt/openinkoopdata_appchallenge/chroma"

loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents([pages[0]])

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [ ]:
query = 'what are the top risks mentioned in the document?'
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# you can use a prompt template
PROMPT_TEMPLATE = """
Antwoord de vraag gebaseerd alleen op basis van de volgende context:
{context}
Beantwoord de vraag op basis van de bovenstaande context: {question}.
Geef een gedetaileerd antwoord.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

model = ChatOpenAI()
response_text = model.predict(prompt)